In [1]:
import pandas as pd

df= pd.read_json('../data_collect/website1/gen_data.json')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2427 entries, 0 to 2426
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          2427 non-null   int64 
 1   name        2427 non-null   object
 2   text        2427 non-null   object
 3   accecpted   2427 non-null   int64 
 4   submission  2427 non-null   int64 
 5   topics      2427 non-null   object
 6   difficulty  2427 non-null   object
 7   link        2427 non-null   object
 8   website     2427 non-null   object
 9   upvotes     2427 non-null   int64 
 10  comments    2427 non-null   int64 
dtypes: int64(5), object(6)
memory usage: 227.5+ KB


## Basic Preprocessing 

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from copy import deepcopy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from spellchecker import SpellChecker
import re
import string

stop_words = set(stopwords.words('english'))
math_pattern = r'\b[\d\+\-\*/=\(\)\s]+\b'
addit= ['given', 'follow', 'exampl', 'consist', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero']
for nw in addit:
	stop_words.add(nw)

stemmer= PorterStemmer()

# Text Preprocessing
def Basic_preprocess(text):
	text = text.lower()
	text = text.translate(str.maketrans('', '', string.punctuation))
  
	# tokenizing the text
	tokens = word_tokenize(text)
	
	# initializing the spell checker
	spell = SpellChecker()
	inco_spell= spell.unknown(tokens);	final_tokens= ""

	# using spell checker to correct the spelling of the word
	for word in tokens:

		if(word in inco_spell):
			new_word= spell.correction(word)
		else:
			new_word= word

		if(new_word not in stop_words ):
			temp= re.sub(r'\d+', '', new_word)
			temp = re.sub(math_pattern, '', temp)

			if(temp!=''):
				new_word= stemmer.stem(word)
				final_tokens+= new_word + " "

	return final_tokens

questions= deepcopy(df)
for i, row in enumerate(questions['text']):
  questions.loc[i, 'text']= Basic_preprocess(row)

[nltk_data] Downloading package stopwords to /home/udayb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/udayb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
questions

,id,name,text,accecpted,submission,topics,difficulty,link,website,upvotes,comments
0,2696,Minimum String Length After Removing Substrings,string consist uppercas english letter appli o...,204900,262700,"String,Stack,Simulation,",Easy,https://leetcode.com/problems/minimum-string-l...,leetcode,926,99
1,1,Two Sum,array integ num integ target return indic numb...,14800000,27400000,"Array,Hash Table,",Easy,https://leetcode.com/problems/two-sum,leetcode,58400,1000
2,2,Add Two Numbers,nonempti link list repres nonneg integ digit s...,5000000,11400000,"Linked List,Math,Recursion,",Medium,https://leetcode.com/problems/add-two-numbers,leetcode,31800,679
3,4,Median of Two Sorted Arrays,sort array nums1 nums2 size respect return med...,2900000,7000000,"Array,Binary Search,Divide and Conquer,",Hard,https://leetcode.com/problems/median-of-two-so...,leetcode,28800,512
4,6,Zigzag Conversion,string paypalishir written zigzag pattern numb...,1500000,3000000,"String,",Medium,https://leetcode.com/problems/zigzag-conversion,leetcode,7900,318
...,...,...,...,...,...,...,...,...,...,...,...
2422,3274,Check if Two Chessboard Squares Have the Same ...,string coordinate1 coordinate2 repres coordin ...,46300,66100,"Math,String,",Easy,https://leetcode.com/problems/check-if-two-che...,leetcode,88,13
2423,3275,K-th Nearest Obstacle Queries,infinit 2d plane posit integ also 2d array que...,30600,64200,"Array,Heap (Priority Queue),",Medium,https://leetcode.com/problems/k-th-nearest-obs...,leetcode,86,19
2424,3276,Select Cells in Grid With Maximum Score,2d matrix grid consist posit integ select cell...,9300,69100,"Array,Dynamic Programming,Bit Manipulation,Mat...",Hard,https://leetcode.com/problems/select-cells-in-...,leetcode,195,23
2425,3277,Maximum XOR Score Subarray Queries,array num integ 2d integ array queri size quer...,3900,8400,"Array,Dynamic Programming,",Hard,https://leetcode.com/problems/maximum-xor-scor...,leetcode,90,14


In [5]:
questions.to_json('preprocessed_data.json')